In [1]:
import os
import PySpin
from IPython import get_ipython
from PIL import Image
import datetime
import matplotlib.pyplot as plt
import sys
import keyboard
import time
import imageio
import math
import numpy as np
import scipy.signal as sig
import cv2
import warnings

In [2]:
# takes the mean of every bin_size columns
def bin_image(im, bin_size):
    shape = im.shape
    while (len(im) % bin_size) != 0:
        im = im[:,:((len(im) // bin_size)*bin_size)]
    new_shape = (im.shape[0], im.shape[1]//bin_size)
    shape = (new_shape[0], im.shape[0] // new_shape[0],
             new_shape[1], im.shape[1] // new_shape[1])
    new_image = im.reshape(shape).mean(-1).mean(1)
    return new_image

In [3]:
def contrast_ratios(image):
    contrast_ratios = []
    for i in range(image.shape[0]):
        row = image[i]
        # calculate max and min pixel value in each row
        row_low = float(np.min(row))#np.quantile(row, 0.1)
        row_high = float(np.max(row))#np.quantile(row, 0.95)     
        # calculate contrast ratio of each row
        contrastratio = (row_high - row_low)/ (2* (row_high+row_low))
        #print(contrastratio)
        contrast_ratios.append(contrastratio)
    return np.mean(contrast_ratios), contrast_ratios

In [4]:
def configure_exposure(cam):
    """
     This function configures a custom exposure time. Automatic exposure is turned
     off in order to allow for the customization, and then the custom setting is
     applied.
    """

    print('*** CONFIGURING EXPOSURE ***\n')

    try:
        result = True

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic exposure. Aborting...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
        print('Automatic exposure disabled...')

        if cam.ExposureTime.GetAccessMode() != PySpin.RW:
            print('Unable to set exposure time. Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        exposure_time_to_set = EXPOSURE_TIME
        exposure_time_to_set = min(cam.ExposureTime.GetMax(), exposure_time_to_set)
        cam.ExposureTime.SetValue(exposure_time_to_set)
        print('Shutter time set to %s us...\n' % exposure_time_to_set)

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [5]:
def reset_exposure(cam):
    """
    This function returns the camera to a normal state by re-enabling automatic exposure.
    """
    try:
        result = True

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable automatic exposure (node retrieval). Non-fatal error...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Continuous)

        print('Automatic exposure enabled...')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [6]:
def configure_gain(cam):

    print('*** CONFIGURING GAIN ***\n')

    try:
        result = True

        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic gain. Aborting...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Off)
        print('Automatic gain disabled...')

    
        if cam.Gain.GetAccessMode() != PySpin.RW:
            print('Unable to set gain . Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        gain_to_set = GAIN
        cam.Gain.SetValue(gain_to_set)
        print('Gain set to %s ...\n' % gain_to_set)

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

In [7]:
def reset_gain(cam):
    try:
        result = True

        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable gain exposure (node retrieval). Non-fatal error...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Continuous)

        print('Automatic gain enabled...')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result 

In [8]:
def setWidth(nodemap,width):
    result = 0
    node_width = PySpin.CIntegerPtr(nodemap.GetNode('Width'))
    if PySpin.IsAvailable(node_width) and PySpin.IsWritable(node_width):
        incrementRemainder = (width - node_width.GetMin()) % node_width.GetInc()
        if width > node_width.GetMax():
            width = node_width.GetMax()
        elif width < node_width.GetMin():
            width = node_width.GetMin()
        elif incrementRemainder !=0:
            width -= incrementRemainder
        node_width.SetValue(width)
        print('Width set to %i...' % node_width.GetValue())
        return 0

    else:
        print('Width not available...')
        return -1 

In [9]:
def setHeight(nodemap,height):
    result = 0
    node_height = PySpin.CIntegerPtr(nodemap.GetNode('Height'))
    if PySpin.IsAvailable(node_height) and PySpin.IsWritable(node_height):
        incrementRemainder = (height - node_height.GetMin())% node_height.GetInc()
        if height > node_height.GetMax():
            height = node_height.GetMax()
        elif height < node_height.GetMin():
            height = node_height.GetMin()
        elif incrementRemainder != 0:
            height -= incrementRemainder
        node_height.SetValue(height)
        print('Height set to %i...' % node_height.GetValue())
        return 0

    else:
        print('Height not available...')
        return -1

In [10]:
continue_recording = True # always True
BIN_SIZE = 8 # bin size for binning image in mode 0
PIXEL_SIZE = .0024 # in mm
LINES_MM = 1 # lines/mm for mode 0
EXPOSURE_TIME = 30000 # in microseconds
CROP_H = 0
GAIN = 0.0
WIDTH = 2500
HEIGHT = 2500
CROP_WIDTH = 600
CROP_HEIGHT = 600
RUN_NUMBER = 23
NUM_ITERATIONS = 29
PLOT_SHOW_TIME = 15 # in seconds

%matplotlib qt
plt.ion()

def handle_close(evt):
    """
    This function will close the GUI when close event happens.

    :param evt: Event that occurs when the figure closes.
    :type evt: Event
    """

    global continue_recording
    continue_recording = False


def acquire_and_display_images(cam, nodemap, nodemap_tldevice):
    """
    This function continuously acquires images from a device and display them in a GUI.

    :param cam: Camera to acquire images from.
    :param nodemap: Device nodemap.
    :param nodemap_tldevice: Transport layer device nodemap.
    :type cam: CameraPtr
    :type nodemap: INodeMap
    :type nodemap_tldevice: INodeMap
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    global continue_recording

    sNodemap = cam.GetTLStreamNodeMap()

    # Change bufferhandling mode to NewestOnly
    node_bufferhandling_mode = PySpin.CEnumerationPtr(sNodemap.GetNode('StreamBufferHandlingMode'))
    if not PySpin.IsAvailable(node_bufferhandling_mode) or not PySpin.IsWritable(node_bufferhandling_mode):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve entry node from enumeration node
    node_newestonly = node_bufferhandling_mode.GetEntryByName('NewestOnly')
    if not PySpin.IsAvailable(node_newestonly) or not PySpin.IsReadable(node_newestonly):
        print('Unable to set stream buffer handling mode.. Aborting...')
        return False

    # Retrieve integer value from entry node
    node_newestonly_mode = node_newestonly.GetValue()

    # Set integer value from entry node as new value of enumeration node
    node_bufferhandling_mode.SetIntValue(node_newestonly_mode)
    
    print('set stream buffer handling mode to newest only.')

    print('*** IMAGE ACQUISITION ***\n')
    try:
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous (enum retrieval). Aborting...')
            return False

        # Retrieve entry node from enumeration node
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(
                node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous (entry retrieval). Aborting...')
            return False

        # Retrieve integer value from entry node
        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

        # Set integer value from entry node as new value of enumeration node
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        print('Acquisition mode set to continuous...')
        setWidth(nodemap, WIDTH)
        setHeight(nodemap, HEIGHT)
        # begin acquisition
        cam.BeginAcquisition()

        print('Acquiring images...')

        # retrieve device serial number
        device_serial_number = ''
        node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
        if PySpin.IsAvailable(node_device_serial_number) and PySpin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()
            print('Device serial number retrieved as %s...' % device_serial_number)

        # Close program
        print('Press (and hold) enter to close the program...')
        
    
        contrast_ratio_means = []
        i = 0
        while(i < NUM_ITERATIONS):
            try:

                #  Retrieve next received image
                #
                #  *** NOTES ***
                #  Capturing an image houses images on the camera buffer. Trying
                #  to capture an image that does not exist will hang the camera.
                #
                #  *** LATER ***
                #  Once an image from the buffer is saved and/or no longer
                #  needed, the image must be released in order to keep the
                #  buffer from filling up.
                
                image_result = cam.GetNextImage(1000)

                #  Ensure image completion
                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d ...' % image_result.GetImageStatus())

                else:
                    # Getting the image data as a numpy array
                    image_data = image_result.GetNDArray()  
                    image_name = "DOF Measurements/dofimage" + str(RUN_NUMBER) + ":" + str(i) + ".png"
                    Image.fromarray(image_data.astype(np.uint8)).save(image_name)

                    #image_data = crop_mag_automatic(image_data, math.floor(image_data.shape[1]*CROP_FACTOR)) 
                                            # math.floor(image_data.shape[0]*CROP_FACTOR)) # not super sure why this first crop is required 
                    
                    # create a FIGURE inside which all data required and image can be viewed side by side 
                    fig = plt.figure(layout='constrained')
                    
                    # separate the figure into two subplots 
                    ax_arrays = fig.subplots(1, 2, squeeze=False)
                    
                    #mag_cropped = crop_mag_automatic(image_data, BIN_SIZE)
                    # take the middle portion of image with size as specified by CROP_WIDTH and CROP_HEIGHT
                    mag_cropped = image_data[image_data.shape[1]//2 - CROP_WIDTH//2:image_data.shape[1]//2 + CROP_WIDTH//2, image_data.shape[0]//2 - CROP_HEIGHT//2 : image_data.shape[0]//2 + CROP_HEIGHT//2]
                    mag_cropped = bin_image(mag_cropped, BIN_SIZE)
                    ax_arrays[0, 0].imshow(mag_cropped, cmap='gray')
                    ax_arrays[0,0].set_title("Cropped and binned image")
                    mean, contrast_ratios_array = contrast_ratios(mag_cropped)
                    contrast_ratio_means.append(mean)
                    ax_arrays[0, 1].plot(contrast_ratios_array)
                    ax_arrays[0, 1].axhline(mean, color='red')
                    ax_arrays[0,1].set_ylabel("Contrast ratios")
                    ax_arrays[0,1].set_title("Contrast ratio per row")
                    plt.pause(PLOT_SHOW_TIME)
                    plot_name = "DOF Measurements/DOF contrast ratios" + str(RUN_NUMBER) + ":" + str(i) + ".png"
                    plt.savefig(plot_name, format = 'png')
                    plt.close()
                        
                    i += 1
                    
                    # If user presses enter, close the program
                    if keyboard.is_pressed('ENTER'):
                        print('Program is closing...')
                        
                        # Close figure
                        plt.close('all')             
                        input('Done! Press Enter to exit...')
                        continue_recording=False  

                #  Release image
                #
                #  *** NOTES ***
                #  Images retrieved directly from the camera (i.e. non-converted
                #  images) need to be released in order to keep from filling the
                #  buffer.
                image_result.Release()

            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                return False
        plt.close()
        contrast_ratio_means = contrast_ratio_means/(np.max(contrast_ratio_means))
        cutoff = contrast_ratio_means > 0.992
        indices = np.nonzero(cutoff)
        lower = np.min(indices)
        higher = np.max(indices)
        print("number of pictures taken that were in focus")
        print(higher - lower + 1)
        plt.plot(contrast_ratio_means)
        plt.axhline(0.992, color = 'black')
        plt.axvline(lower, color = 'green')
        plt.axvline(higher, color = 'green')
        plt.ylabel("Average Contrast Ratios")
        plt.pause(PLOT_SHOW_TIME)
        plot_name = "DOF Measurements/Depth of field" + str(RUN_NUMBER) + ".png"
        plt.savefig(plot_name, format = 'png')
        plt.close()
        #  End acquisition
        #
        #  *** NOTES ***
        #  Ending acquisition appropriately helps ensure that devices clean up
        #  properly and do not need to be power-cycled to maintain integrity.
        cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return True


def run_single_camera(cam):
    """
    This function acts as the body of the example; please see NodeMapInfo example
    for more in-depth comments on setting up cameras.

    :param cam: Camera to run on.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        nodemap_tldevice = cam.GetTLDeviceNodeMap()

        # Initialize camera
        cam.Init()
        
        nodemap = cam.GetNodeMap()

        if not configure_gain(cam):
            return False
        if not configure_exposure(cam):
            return False
        
        # cam.SensorShutterMode.SetValue(PySpin.SensorShutterMode_Rolling)
        # cam.GammaEnable.SetValue(False)
        # cam.PixelFormat.SetValue(PySpin.PixelFormat_Mono16)
        
        # Retrieve GenICam nodemap        
        # Acquire images
        result &= acquire_and_display_images(cam, nodemap, nodemap_tldevice)
        
        # Reset exposure
        result &= reset_exposure(cam)
        result &= reset_gain(cam)
        
        # Deinitialize camera
        cam.DeInit()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result


def main():
    """
    Example entry point; notice the volume of data that the logging event handler
    prints out on debug despite the fact that very little really happens in this
    example. Because of this, it may be better to have the logger set to lower
    level in order to provide a more concise, focused log.

    :return: True if successful, False otherwise.
    :rtype: bool
    """
    result = True
    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()

    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:

        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):
        print('Running example for camera %d...' % i)

        result &= run_single_camera(cam)
        print('Camera %d example complete... \n' % i)

    # Release reference to camera
    # NOTE: Unlike the C++ examples, we cannot rely on pointer objects being automatically
    # cleaned up when going out of scope.
    # The usage of del is preferred to assigning the variable to None.
    del cam

    # Clear camera list before releasing system
    cam_list.Clear()


    # Release system instance
    system.ReleaseInstance()

    return result

if __name__ == '__main__':
    if main():
        sys.exit(0)
    
    else:
        sys.exit(1)

Library version: 2.7.0.128
Number of cameras detected: 0
Not enough cameras!


Done! Press Enter to exit... 


SystemExit: 1

/Users/karolyn/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
